In [1]:
import pandas as pd
import mysql.connector #package: mysql-connector-python

In [32]:
df = pd.read_csv("/home/hadoop/Desktop/SharedFolder/Measurement_info.csv",sep=",")
df.head()

,Measurement date,Station code,Item code,Average value,Instrument status
0,2017-01-01 00:00,101,1,0.004,0
1,2017-01-01 00:00,101,3,0.059,0
2,2017-01-01 00:00,101,5,1.200,0
3,2017-01-01 00:00,101,6,0.002,0
4,2017-01-01 00:00,101,8,73.000,0


In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3885066 entries, 0 to 3885065
Data columns (total 5 columns):
 #   Column             Dtype  
---  ------             -----  
 0   Measurement date   object 
 1   Station code       int64  
 2   Item code          int64  
 3   Average value      float64
 4   Instrument status  int64  
dtypes: float64(1), int64(3), object(1)
memory usage: 148.2+ MB


In [35]:
df.columns = [col.lower().replace(" ", "_") for col in list(df.columns)]

In [36]:
for col in df.columns:
    print(col, ": ", df[col].unique())
# df.unique()

measurement_date :  ['2017-01-01 00:00' '2017-01-01 01:00' '2017-01-01 02:00' ...
 '2019-12-31 21:00' '2019-12-31 22:00' '2019-12-31 23:00']
station_code :  [101 102 103 104 105 106 107 108 109 110 111 112 113 114 115 116 117 118
 119 120 121 122 123 124 125]
item_code :  [1 3 5 6 8 9]
average_value :  [4.00e-03 5.90e-02 1.20e+00 ... 5.50e+00 1.72e+01 1.76e-01]
instrument_status :  [0 9 4 8 1 2]


In [25]:
print(df["average_value"].max(), df["average_value"].min())

6256.0 -1.0


In [ ]:
# part zero: prep the schema with sqlalchemy

In [22]:
from sqlalchemy import create_engine, MetaData, Table, Column
from sqlalchemy import Integer, Numeric, String, ForeignKey, DateTime

In [38]:
# a minimal schema
metadata = MetaData()

schema_station = Table('station', metadata,
    Column('measurement_date', DateTime() ),
    Column('station_code', Integer()),
    Column('item_code', Integer()),
    Column('average_value', Numeric(8, 3)),
    Column('instrument_status', Integer()),
)

In [54]:
engine_memory = create_engine('sqlite://')

metadata.create_all(engine_memory)

In [56]:
df.to_sql("station", con=engine_memory, index=False, if_exists="replace")

In [58]:
# check schema
sql_str = """
SELECT sql FROM sqlite_master WHERE name='station';  
"""
results = pd.read_sql(sql_str, engine_memory)
results.sql.values

array(['CREATE TABLE station (\n\tmeasurement_date TEXT, \n\tstation_code BIGINT, \n\titem_code BIGINT, \n\taverage_value FLOAT, \n\tinstrument_status BIGINT\n)'],
      dtype=object)

In [49]:
sql_str = """
SELECT sql FROM sqlite_master WHERE name='station1';  
"""
results = pd.read_sql(sql_str, engine_memory)
results.sql.values

array(['CREATE TABLE station1 (\n\tmeasurement_date TEXT, \n\tstation_code BIGINT, \n\titem_code BIGINT, \n\taverage_value FLOAT, \n\tinstrument_status BIGINT\n)'],
      dtype=object)

In [41]:
sql_str = """
select * from station
"""
results = pd.read_sql(sql_str, engine_memory)
results.head()

,index,measurement_date,station_code,item_code,average_value,instrument_status
0,0,2017-01-01 00:00,101,1,0.004,0
1,1,2017-01-01 00:00,101,3,0.059,0
2,2,2017-01-01 00:00,101,5,1.200,0
3,3,2017-01-01 00:00,101,6,0.002,0
4,4,2017-01-01 00:00,101,8,73.000,0


In [42]:
results.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3885066 entries, 0 to 3885065
Data columns (total 6 columns):
 #   Column             Dtype  
---  ------             -----  
 0   index              int64  
 1   measurement_date   object 
 2   station_code       int64  
 3   item_code          int64  
 4   average_value      float64
 5   instrument_status  int64  
dtypes: float64(1), int64(4), object(1)
memory usage: 177.8+ MB


In [4]:
# part I: mysql-connector-python 
# https://dev.mysql.com/doc/connector-python/en/

In [5]:
# prep for connection
host = "sql450.main-hosting.eu"
db = "u820202695_dev1"
user = "u820202695_root"
passwd = "Root0000"###SHOULD BE KEPT HIGHLY SECRETIVE!!!!

In [6]:
mydb = mysql.connector.connect(
  host=host,
  user=user,
  password=passwd,
  database=db
)

mycursor = mydb.cursor(dictionary=True) # allow return column names

mycursor.execute("SHOW TABLES")

myresult = mycursor.fetchall()

for x in myresult:
  print(x)

mycursor.close()
mydb.close()

{'Tables_in_u820202695_dev1': 'first_test'}


In [7]:
# note the connect need to establish everytime when execute a query
# write an wrapper for the connection init
def mysql_executor(conn_para, sql_str):

    mydb = mysql.connector.connect(**conn_para)
    mycursor = mydb.cursor(dictionary=True) 
    mycursor.execute(sql_str)
    myresult = mycursor.fetchall()
#     for x in myresult:
#       print(x)

    mycursor.close()
    mydb.close()
    return myresult

# test #1   
para_dict = dict(host = "sql450.main-hosting.eu",
                 db = "u820202695_dev1",
                 user = "u820202695_root",
                 passwd = "Root0000")
result = mysql_executor(para_dict, "SHOW TABLES")
print(result)
pd.DataFrame(result)
# mysql_connector()

[{'Tables_in_u820202695_dev1': 'first_test'}]


,Tables_in_u820202695_dev1
0,first_test


In [8]:
# test #2
sql_str = """
select * from first_test
limit 2;
"""
result = mysql_executor(para_dict, sql_str)
pd.DataFrame(result)

,recipe_id,recipe_name
0,3,Grilled Cheese
1,1,Tacos


In [9]:
# part II use sqlalchemy

# ref: SQLAlchemy 1.4 Documentation: MySQL and MariaDB
# https://docs.sqlalchemy.org/en/14/dialects/mysql.html

In [10]:
from sqlalchemy import create_engine
import pymysql 
#https://anaconda.org/anaconda/pymysql

# pymysql is light-weighted comapring to mysql-connector-python
# we add another layer of abstraction (sqlalchemy) to handle error and transaction

In [11]:
host = "sql450.main-hosting.eu"
db = "u820202695_dev1"
user = "u820202695_root"
passwd = "Root0000"

sql_str = """
select * from first_test
limit 2;
"""

# the syntax is mysql+pymysql://<user>:<password>@<host>[:<port>]/<dbname>
conn_para = f"mysql+pymysql://{user}:{passwd}@{host}/{db}" 
engine = create_engine(conn_para, pool_recycle=3600, pool_pre_ping=True)
my_data = pd.read_sql(sql_str,engine)
my_data

,recipe_id,recipe_name
0,3,Grilled Cheese
1,1,Tacos


In [12]:
# By default, MySQL closes connections idle for more than eight
# hours. To work around this issue, use pool_recycle=3600 when
# creating an engine, as shown in the example.

# The create_engine function returns an instance of an engine;
# however, it does not actually open a connection until an action is
# called that would require a connection, such as a query.
# In other words, we don't have establish connection everytime we query


sql_str = """
select * from first_test
where recipe_id=2
limit 3;
"""
# engine = create_engine(conn_para, pool_recycle=3600, pool_pre_ping=True)
results = engine.execute(sql_str)
# results.fetchall()
# results.keys()
pd.DataFrame(data=results.fetchall(), columns=results.keys())


,recipe_id,recipe_name
0,2,Tomato Soup


In [14]:
# retieve schema (method 1)
from sqlalchemy import MetaData, Table
metadata = MetaData()
table = Table('first_test', metadata, autoload_with=engine)
table

Table('first_test', MetaData(), Column('recipe_id', INTEGER(display_width=11), table=<first_test>, primary_key=True, nullable=False), Column('recipe_name', VARCHAR(collation='utf8mb4_unicode_ci', length=30), table=<first_test>, nullable=False), schema=None)

In [15]:
# retrieve schema method 2 (recommanded)
metadata = MetaData()
metadata.reflect(bind=engine)
# metadata.create_all(engine)

show_tables = metadata.tables.keys()
show_tables

# view schema of the "1st" table
table_1st = list(show_tables)[0]
Table(table_1st, metadata, autoload_with=engine).columns.values()

[Column('recipe_id', INTEGER(display_width=11), table=<first_test>, primary_key=True, nullable=False),
 Column('recipe_name', VARCHAR(collation='utf8mb4_unicode_ci', length=30), table=<first_test>, nullable=False)]

In [16]:
# verify using legacy sql method
sql_str = """
describe first_test;
"""
results = engine.execute(sql_str)
# results.fetchall()
# results.keys()
pd.DataFrame(data=results.fetchall(), columns=results.keys())

,Field,Type,Null,Key,Default,Extra
0,recipe_id,int(11),NO,PRI,None,
1,recipe_name,varchar(30),NO,UNI,None,


# SQLAlchemy Essential

In [ ]:
# Types
# ....


In [17]:
# Metadata
# Metadata is used to tie together the database structure so it can be quickly accessed
# inside SQLAlchemy.
from sqlalchemy import MetaData
metadata = MetaData()

In [18]:
# Tables
# Table objects are initialized in SQLAlchemy Core in a supplied MetaData object by
# calling the Table constructor with the table name and metadata; any additional arguments
# are assumed to be column objects.

from sqlalchemy import Table, Column, Integer, Numeric, String, ForeignKey

# example 1
cookies = Table('cookies', metadata,
Column('cookie_id', Integer(), primary_key=True),
Column('cookie_name', String(50), index=True),
Column('cookie_recipe_url', String(255)),
Column('cookie_sku', String(55)),
Column('quantity', Integer()),
Column('unit_cost', Numeric(12, 2))
)

In [19]:
ins = cookies.insert().values(
cookie_name="chocolate chip",
cookie_recipe_url="http://some.aweso.me/cookie/recipe.html",
cookie_sku="CC01",
quantity="12",
unit_cost="0.50"
)
print(str(ins))

INSERT INTO cookies (cookie_name, cookie_recipe_url, cookie_sku, quantity, unit_cost) VALUES (:cookie_name, :cookie_recipe_url, :cookie_sku, :quantity, :unit_cost)


# what's next

- [Transaction Isolation Level](https://docs.sqlalchemy.org/en/14/dialects/mysql.html#transaction-isolation-level)
